# Using Mappings as Inputs

<a href="https://colab.research.google.com/github/DeepTrackAI/deeplay/blob/develop/tutorials/advanced-topics/AT201_mappings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install deeplay  # Uncomment if running on Colab/Kaggle.

`DeeplayModule` objects can take mappings—such as Python dictionaries—as inputs, making it easy to define and manage complex data processing pipelines. By using mappings, you can organize and access data flexibly, allowing models to handle a wide range of inputs seamlessly. This is especially useful when working with multiple input types, customizing sequential data processing, or setting up parallel pipelines.

## Introduction to Mappings in Deeplay

When using mappings as inputs to `DeeplayModule` objects, you define the relationship between your input and output data structures by configuring the `.set_input_map()` and `.set_output_map()` methods of the module. For instance, you can clearly indicate which input features feed into specific layers or operations, ensuring that your model processes data correctly and efficiently.

For example, this code defines a linear module that takes a dictionary with the key `x` as input and produces an output stored under a different key, `y`.

In [1]:
import deeplay as dl
import torch

module = dl.Layer(torch.nn.Linear, 10, 64)

module.set_input_map("x")
module.set_output_map("y")

Layer[Linear](in_features=10, out_features=64)

If we set the output key to be the same as the input key, we can define a linear module that takes a dictionary with the key `x` as input and overwrites the value associated with `x` with the output of the linear transformation.

In [2]:
module = dl.Layer(torch.nn.Linear, 10, 64)

module.set_input_map("x")
module.set_output_map("x")

Layer[Linear](in_features=10, out_features=64)

Once the module is built, the input data can be passed to the module as with `Tensor` objects. The module will automatically map the input and output data to the expected data structure.

In [3]:
input_data = {
    'x': torch.randn(20, 10),
}

built = module.build()

output_data = built(input_data)

print("output keys: ", output_data.keys())
print("input x shape: ", input_data['x'].shape)
print("output x shape: ", output_data['x'].shape)

output keys:  dict_keys(['x'])
input x shape:  torch.Size([20, 10])
output x shape:  torch.Size([20, 64])


Currently supported mapping objects include dictionaries, such as ... 

In [4]:
input_data = {
    'x': torch.randn(20, 10),
}

... and Torch Geometric Data objects, like the following:

In [5]:
from torch_geometric.data import Data

input_data = Data(
    x=torch.randn(3, 10),
)

## Basic Operations with Mappings

The `.set_input_map()` method allows for local key re-assignment. This is particularly useful when the keys in the input data do not align with the keys expected by the module. By using this method, you can explicitly define how your input data should be mapped to the keys that the module will use.

In the following example, we create a multi-head attention module, which expects inputs in the form of query, key, and value tensors. Here, we use the same input, `x`, for all three inputs—query, key, and value.This demonstrates how to configure the mappings to align with the module's requirements, even if the input data is organized differently.

In [6]:
module = dl.Layer(torch.nn.MultiheadAttention, embed_dim=10, num_heads=2)

module.set_input_map(query="x", key="x", value="x")

Layer[MultiheadAttention](embed_dim=10, num_heads=2)

Additionally, we can configure the output mapping of the multi-head attention module. Using the `.set_output_map()` method, we define the ouput mappings, where the attention output is mapped to `x` and the attention weights are mapped to `attention_weights`:

In [7]:
module.set_output_map("x", "attention_weights")

Layer[MultiheadAttention](embed_dim=10, num_heads=2)

Alternatively, we can specify the output mappings using their indices. For example:

In [8]:
module.set_output_map(x=0, attention_weights=1)

Layer[MultiheadAttention](embed_dim=10, num_heads=2)

In this case, `x=0` and `attention_weights=1` indicate that the output dictionary will assosiate the first output of the module with `x` and the second output with `attention_weights`.

In [9]:
built = module.build()

input_data = Data(x=torch.randn(100, 10))

output_data = built(input_data)

print("output keys: ", output_data.keys())
print("output x shape: ", input_data.x.shape)
print("output attention_weights shape: ", output_data.attention_weights.shape)

output keys:  ['x', 'attention_weights']
output x shape:  torch.Size([100, 10])
output attention_weights shape:  torch.Size([100, 100])


You can also add new keys to the output data structure to store intermediate results:

In [10]:
module = dl.Layer(torch.nn.MultiheadAttention, embed_dim=10, num_heads=2)

module.set_input_map(query="x", key="x", value="x")

module.set_output_map(
    x=0,
    intermediate_x=0,
    attention_weights=1,
    intermediate_attention_weights=1,
)

Layer[MultiheadAttention](embed_dim=10, num_heads=2)

The first output of the module is mapped to both `x` and `intermediate_x`, allowing you to store the same result in two different keys. Similarly, the second output is mapped to both `attention_weights` and `intermediate_attention_weights`.

In [11]:
built = module.build()

input_data = Data(x=torch.randn(100, 10))

output_data = built(input_data)

print("output keys: ", output_data.keys())

print("output x shape: ", input_data.x.shape)
print("output intermediate_x shape: ", output_data.intermediate_x.shape)

print("output attention_weights shape: ", output_data.attention_weights.shape)
print(
    "output intermediate_attention_weights shape: ",
    output_data.intermediate_attention_weights.shape,
)

output keys:  ['x', 'attention_weights', 'intermediate_x', 'intermediate_attention_weights']
output x shape:  torch.Size([100, 10])
output intermediate_x shape:  torch.Size([100, 10])
output attention_weights shape:  torch.Size([100, 100])
output intermediate_attention_weights shape:  torch.Size([100, 100])


## Mappings Allow Sequential Processing

Mappings enable the sequencing of branched pipelines, giving you full control over how data flows through each module. This allows you to fully exploit the modular design of `DeeplayModule` objects. For instance:

In [ ]:
from torch_geometric.nn import LayerNorm

module = dl.Sequential(
    dl.Layer(torch.nn.Bilinear, 10, 10, 10) \
        .set_input_map("x1", "x2").set_output_map("y"),
    dl.Layer(LayerNorm, 10) \
        .set_input_map(x="y", batch="batch").set_output_map("y"),
    dl.Add().set_input_map("x1", "y").set_output_map("x1"),
    dl.Add().set_input_map("x2", "y").set_output_map("x2"),
)

In the code above:

1. The first layer combines inputs `x1` and `x2` unsig a bilinear operation, producing an output labeled `y`. 

2. Next, a layer normalization is applied to `y`, using batch information from the `batch` input. The result is again assigned to `y`. 

3. Finally, `x1` and `x2` are independently summed with `y`. Each of these sums is stored back into their respective keys `x1` and `x2`, thereby implementing a residual connection.


In [13]:
built = module.build()

input_data = Data(
    x1=torch.randn(5, 10),
    x2=torch.randn(5, 10),
    batch=torch.Tensor([0, 0, 0, 1, 1]).long(),
)

output_data = built(input_data)

print("output keys: ", output_data.keys())
print("output x1 shape: ", output_data.x1.shape)
print("output x2 shape: ", output_data.x2.shape)
print("output y shape: ", output_data.y.shape)

output keys:  ['y', 'x2', 'batch', 'x1']
output x1 shape:  torch.Size([5, 10])
output x2 shape:  torch.Size([5, 10])
output y shape:  torch.Size([5, 10])


## Defining Parallel Pipelines

With Deeplay's `Parallel` objects, you can create piplines that process input mapping objects concurrently. Each module in the your pipeline will receive the input data, extract the relevant keys, and handle the processing independently. Finally, the outputs from each module are stored as separate keys within a single mapping object, allowing you to access each result individually.

In [14]:
module = dl.Parallel(
    dl.Layer(torch.nn.Linear, 10, 30).set_input_map("x").set_output_map("y"),
    dl.Layer(torch.nn.Linear, 10, 30).set_input_map("x").set_output_map("z"),
)

built = module.build()

input_data = Data(x=torch.randn(5, 10))

output_data = built(input_data)

print("output keys: ", output_data.keys())
print("output y shape: ", output_data.y.shape)
print("output z shape: ", output_data.z.shape)

output keys:  ['x', 'y', 'z']
output y shape:  torch.Size([5, 30])
output z shape:  torch.Size([5, 30])


In this example, both linear layers take `x` as input, producing outputs labeled `y` and `z`, respectively.

Alternatively, you can specify the output mappings directly within `Parallel`'s constructor. In the example below, each module is assigned a key (`y` and `z` for the first and second layer, respectively), and the output mappings are automatically determined based on these keys:

In [15]:
module = dl.Parallel(
    y=dl.Layer(torch.nn.Linear, 10, 30).set_input_map("x"),
    z=dl.Layer(torch.nn.Linear, 10, 30).set_input_map("x"),
)

This method streamlines the syntax, but offers less explicit control over output mappings compared to configuring each module individually.

## Using Mappings with Graph Data

Deeplay’s graph operations let you input mapping objects directly, simplifying the management of graph data. In graphs, nodes (entities) are connected by edges (relationships), and Deeplay’s mappings makes it easy to work with these inputs. 

For example, in a graph convolutional neural network (GCN), node features are updated based on information from neighboring nodes. In this example, `x` represents the features of three nodes, while `edge_index` defines the connections between nodes. Deeplay’s GCN layer automatically reads these mappings, processes each node’s features while considering its neighboring nodes, and stores the result back in the mapping for easy access.

In [ ]:
gcn = dl.components.gcn.GraphConvolutionalNeuralNetwork(
    in_features=10, hidden_features=[], out_features=10,
)

built_gcn = gcn.build()

input_data = Data(
    x=torch.randn(3, 10),  # Node features for 3 nodes, each with 10 features.
    edge_index=torch.tensor(
        [
            [0, 1, 1, 2], 
            [1, 0, 2, 1]
        ]
    ),  # Graph connectivity, indicating edges between nodes.
)

output_data = built_gcn(input_data)

print("output keys: ", output_data.keys())

output keys:  ['x', 'edge_index']


## From Mappings to Tensor Objects

Deeplay supports a seamless transition between mapping objects and Tensor objects. The output data structure can be converted to a Tensor object using the `FromDict` module:

In [ ]:
module = dl.Sequential(
    dl.Layer(torch.nn.Bilinear, 10, 10, 10) \
        .set_input_map("x1", "x2").set_output_map("y"),
    dl.Layer(LayerNorm, 10) \
        .set_input_map(x="y", batch="batch").set_output_map("y"),
    dl.FromDict("y"), 
    dl.Layer(torch.nn.Linear, 10, 1)
)

In [18]:
built = module.build()

input_data = Data(
    x1=torch.randn(5, 10),
    x2=torch.randn(5, 10),
    batch=torch.Tensor([0, 0, 0, 1, 1]).long(),
)

output_data = built(input_data)

print("output type: ", type(output_data))
print("output shape: ", output_data.shape)

output type:  <class 'torch.Tensor'>
output shape:  torch.Size([5, 1])
